# Creating a dictionary of dataset version selection for a series of variables and experiments, with one version per model variant showing enough data.
# This notebook has yet to be executed manually, with some iteration
## The structure of the dictionary (written as a json file) is 
data_versions[experiment][variable][table][model][variant]=(grid,version,data_period)
## and it includes all models which have (enough) data for the experiment and variable


## The selection is done for each experiment separately.
## In order to get the same variant for variables pr and evspsbl, a second pass is done where, for some models, the variant is imposed

### Stéphane Sénési

In [1]:
# Climaf version >= 1.2.13 (see https://climaf.readthedocs.io)
climaf_lib     = "/home/ssenesi/climaf_installs/running" 
# AR6/WGI/chapter8 CliMAF-based package
CAMMAC            = "/home/ssenesi/CS2"

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
import datetime, glob, json, sys, copy
#
sys.path.append(climaf_lib) 
from climaf.api import *
from climaf.cache import stamping
climaf.cache.stamping=False

In [6]:
sys.path.append(CAMMAC) # Assume AR6 figures project packages are one level up
from CAMMAClib.ancillary  import feed_dic
from CAMMAClib.mips_et_al import institute_for_model, mip_for_experiment, models_for_experiments, read_versions_dictionnary

In [7]:
import CAMMAClib.mips_et_al
reload(CAMMAClib.mips_et_al)

<module 'CAMMAClib.mips_et_al' from '/home/ssenesi/CS2/CAMMAClib/mips_et_al.pyc'>

In [8]:
def list_models_and_institute() :
    global institutes
    institutes=dict()
    pat="/bdd/CMIP6/CMIP/*/*"
    alls=glob.glob(pat)
    i=dict()
    models=set()
    for e in alls :
        m=e.split("/")[5]
        models.add(m)
        i[m]=e.split("/")[4]
    for m in sorted(list(models)):
        print "%-20s %s"%(m,i[m])
        institutes[m]=i[m]
#
#list_models_and_institute()

In [9]:
def print_models_with_more_than_one_grid_for_sos() :
    from CAMMAClib.mips_et_al import models_data
    import glob


    for m in models_data :
        p=models_data[m][0]
        if p != "CMIP6" :
            continue
        i=models_data[m][1]
        pat="/bdd/CMIP6/CMIP/%s/%s/historical/*"%(i,m)
        reals=glob.glob(pat)
        if len(reals) > 0 :
            grids=set()
            for r in reals :
                grs=glob.glob("%s/Omon/sos/*"%r)
                #grs=glob.glob("%s/day/pr/*"%r)
                for g in grs :
                    gg=g.split("/")[10]
                    grids.add(gg)
            if len(list(grids)) > 1:
                print "%-20s "%m,list(grids)
        else :
            print "No historical data for =",m

#print_models_with_more_than_one_grid_for_sos()

In [10]:
# For models providing some variables in multiple grids, choose one (actually used for some ocean variables, as "sos", and for GFDL-CM4 for sos and pr_day)
preferred_grids={
    "CESM2-WACCM":"gn","CESM2-WACCM-FV":"gn","CESM2-WACCM-FV2":"gn","CESM2-FV2":"gn","CESM2":"gn",
    "CNRM-CM6-1":"gr1","CNRM-ESM2-1":"gr1",
    "GFDL-ESM4":"gn","GFDL-CM4":["gn","gr1"], 
    "IPSL-CM6A-LR" : "gr1", 
    "MRI-ESM2-0" : "gn"}

In [11]:
def list_CMIP6_models_that_run(experiment):
    import glob
    print "Identifying models that ran %s"%experiment
    mip=mip_for_experiment(experiment)
    exps=glob.glob("/bdd/CMIP6/%s/*/*/%s"%(mip,experiment))
    models = [ e.split("/")[5] for e in exps ]
    return models

    # Select lists of attributes having some relevant data
    #clog('error')
    #d=ds(project=project, experiment=experiment,model="*", institute="*",
    #                      period=period, variable=variable, table=table, 
    #                      version="*", mip=mip_for_experiment(experiment),realization="*" )
    #return d.explore('choices')['model']
    

In [28]:
piControl_models=list_CMIP6_models_that_run("piControl") 
print len(piControl_models)
for m in piControl_models : 
    print m
for m in piControl_models : 
    try :
        i=institute_for_model(m)
    except :
        print("Must document institute for %s"%model)
    

Identifying models that ran piControl
57


In [12]:
def select_all_attributes_combinations_with_data(all_cases,resolve_issues,model,experiment,variable,table,period,project="CMIP6",imposed_realization=None):
    """
    Feed all_cases[MODEL][GRID][REALIZATION][VERSION] with the corresponding dataset objects for VARIABLE in PROJECT 
    and EXPERIMENT, ensuring PERIOD is covered by data files, and complying with IMPOSED_REALIZATION (if not None)
    
    Will feed entries for various grids, versions and realizations (provided data exist)
    
    Also feed dic resolve_issues with datasets showing a problem at 'resolve' stage (usually holes in data coverage)
    """
    d=ds(project=project, experiment=experiment,model=model, institute="*",
                          period=period, variable=variable, table=table, 
                          version="*", mip=mip_for_experiment(experiment),realization="*" )
    choices=d.explore('choices')
    if len(choices)==0 : return
    #print model,experiment,choices
    grids=choices['grid']
    if type(grids)!= type([]) : grids=[grids]
    #
    versions=choices['version']
    if type(versions)==type([]) : 
        if 'latest' in versions : versions.remove('latest')
    else : versions=[versions]
    #
    if imposed_realization is not None :
        if imposed_realization not in choices['realization'] :
            raise ValueError("Imposed realization %s not found for %s %s %s in %s"%(imposed_realization,model,experiment,variable,choices['realization']))
        else:
            realizations=[imposed_realization]
    else: 
        realizations=choices['realization']
        if type(realizations) != type([]) : realizations=[realizations]
    #
    got_some=False
    for g in grids :
        for v in versions:
            for r in realizations :
                    d=ds(project=project, experiment=experiment,model=model, institute=institute_for_model(model),
                              period=period, variable=variable, table=table, 
                              version=v, mip=mip_for_experiment(experiment),realization=r, grid=g )
                    try :
                        d=d.explore('resolve')
                        # Not all combinations or grid, realization and version have data
                        if d.baseFiles() is not None:
                            feed_dic(all_cases,d,experiment,variable,table,model,g,r,v)
                            got_some=True
                    except :
                        #print "--> Issue (holes?) for ",model, experiment, variable, table, g,r,v
                        feed_dic(resolve_issues,`d`,model,g,r,v)
                        continue
    return got_some

In [13]:
def select_realizations_and_version(cases,selection,coverage_issues,model,experiment,variable,table,period,project): 
    """
    Scrutinize all datasets in cases[MODEL][GRID][REALIZATION][VERSION], selecting the prefered 
    grid for MODEL (if applicable), then looping across realization and their versions for identifying 
    one version per realization (with prioriy to the newest) which datafiles covers PERIOD
    
    The result is stored in SELECTION[MODEL][REALIZATION] as a triplet (grid,version,covered_data_period)
    
    Pairs with uncomplete data coverage are stored in COVERAGE_ISSUES, but only until (in each loop 
    on versions) a correct version is found
    """
    # all combinations which data files actually covers the requested period
    # with one single grid+realization+version per model
    grids=cases[experiment][variable][table][model].keys()
    if len(grids) > 1 : 
        gr=None
        if model in preferred_grids : 
            gr=preferred_grids[model]
            if type(gr)==type([]) :
                gr=set(gr).intersection(set(grids))
                if len(gr)==1 :
                    gr=gr.pop()
                else :
                    gr=None
        if gr is None or gr not in grids :
            print "!!! multiple grids for ",model,experiment, variable, table, ":",grids
            raise ValueError("!!! multiple grids for %s %s %s %s : %s"%(model,experiment, variable, table,grids))
    else :
        gr=grids[0]
    reals=cases[experiment][variable][table][model][gr].keys()
    found_one_realization=False
    for real in reals :
        ok=False
        versions=cases[experiment][variable][table][model][gr][real].keys()
        versions.sort()
        sversions=[ v for v in versions ]
        while len(versions)>0 and not ok :
            holes=dict()
            last=versions[-1]
            # Check available period is OK 
            d=ds(project=project, experiment=experiment,model=model, institute=institute_for_model(model),
                          period="*", variable=variable, table=table, 
                          version=last, mip=mip_for_experiment(experiment),realization=real, grid=grids[0] )
            data_period=d.explore('choices',operation='union').get('period',None)
            speriod=data_period
            #print "model=",model,"data_period=",data_period,type(data_period),type(data_period[0])
            if data_period is not None : 
                data_period=data_period[0]
                if period != "*" :
                    if data_period.includes(climaf.classes.init_period(period)) :
                        ok=True
                        actual_period=data_period
                    else : 
                        holes[last]=`speriod`
                else :
                    if experiment != "piControl" :
                        raise ValueError("Cannot process period * except for piControl")
                    # we relax the AR5 criterion of 100 years for spinup followed by 400 years
                    # by suppressing the 100 years criterion, 
                    if (data_period.end - data_period.start) >= 400*datetime.timedelta(days=365) :
                        ok=True
                        actual_period=data_period
                    else : 
                        holes[last]=`speriod`
            else :
                holes = None
            if not ok :
                #print "*** %-20s : issue with %s , no version covers the period (%s); "%(model,srindex, holes)
                feed_dic(coverage_issues,holes,model,gr,real,last)
                versions.remove(last)
        if ok :
            found_one_realization=True
            #print "%-20s %s: use %s (among %s : %s "%(model,real, last, len(sversions),sversions)
            feed_dic(selection,(grids[0],last,data_period.pr()),model,real)
            #return(grids[0],srindex,last,data_period.pr())
        else :
            print "%-20s %s: no valid version among %s : %s "%(model,real, len(sversions),sversions)
    if found_one_realization is False :
        print "no valid realization for ",model,experiment,variable,table,period
    else :
        return True


In [14]:
#clog('error')
#c,ok,ri,ci,noreal=select_models_data_versions("ssp585",variable="pr",table="Amon",period=None)#,models=["FGOALS-f3-L"])

## Function for naming and writing versions dictionnary

In [15]:
def write_versions_dic(d,tag) :
    jsfile="Data_versions_selection_%s.json"%tag
    with open(jsfile,"w") as f :
        json.dump(d,f,separators=(',', ': '),indent=3,ensure_ascii=True)
    print "Data versions dictionnary written as "+jsfile

#write_versions_dic("201902071814")

In [16]:
def read_versions_dic(tag) :
    jsfile="Data_versions_selection_%s.json"%tag
    with open(jsfile,"r") as f :
        rep=json.load(f)
    return rep
def read_all_versions_dic(tag):
    select        = read_versions_dic(tag)  
    all_holes     = read_versions_dic(tag+"_holes")  
    all_coverages = read_versions_dic(tag+"_resolve")  
    noreals       = read_versions_dic( tag+"_noreals")  
    return select, all_versions, all_holes, all_coverages, noreals
#dics=read_all_versions_dic("20200529t")

In [17]:
def select_models_data_versions(experiment,variable="pr",table="Amon",period=None,imposed_realizations={},models=None,do_select=True):
    """
    For CMIP6 model data, identify models which ran EXPERIMENT (or use provided list of models) 
    and which produced VARIABLE in TABLE.
    
    For each realization (or for the single realization indicated by each model in dict imposed_realization), 
    identify latest data version which covers PERIOD (which defaults to experiment duration, or 400 years 
    for piControl)
    
    IMPOSED_REALIZATIONS is a dict with keys=model, or an empty dict for not using the feature
    
    4 dics and a list are returned :
    
      - cases[experiment][variable][table][model][grid][realization][version] = the corresponding dataset object 
        for all available datasets
        
      - selection[model][realization]=(grid,version,period) provides the selected version and grid, and documents 
        the period, for all datasets with enough data and selecting the latest available version
        
      - holes[model][grid]realization][version] = printable description of the data holes for each combination 
        showing some
      
      - coverage_issues[model][grid]realization][version] = printable description of the available data period 
        for those case where this period is too short
      
      - no_real = list of models with no valid realization for that variable and experiment
      
    """
    print
    print "Processing experiment %s, variable %s, table %s"%(experiment,variable,table)
    print 50*"*"
    if models is None :
        models=list_CMIP6_models_that_run(experiment)
    #    
    if period is None :
        if experiment=="historical" :
            period="18500101-20141230"  # Do not include 20141231 thanks to the british guys !
        elif experiment[0:3]=="ssp":
            period="2015-2099"  #should end at 2100, but, at some date, CAMS-1 did not produce last year
        elif experiment=="piControl":
            period="*"
        else :
            raise ValueError("cannot guess period for experiment %s"%experiment)
    #
    cases=dict()
    selection=dict()
    holes=dict()
    coverage_issues=dict()
    no_real=[]
    project="CMIP6"
    #
    for model in models :
        #print model,variable,table
        imposed=imposed_realizations.get(model,None)
        if experiment=="piControl" : imposed=None
        #
        got_some=select_all_attributes_combinations_with_data(cases,holes,model,experiment,variable,table,period,project,imposed)
        #print cases
        if do_select :
            ok=None
            if got_some and model in cases[experiment][variable][table] :
                #ok=select_realization_and_version(cases,selection,coverage_issues,model,experiment,variable,table,period,project)
                ok=select_realizations_and_version(cases,selection,coverage_issues,model,experiment,variable,table,period,project)
            if ok is None :
                no_real.append(model)
    print "models with no realization showing complete data for %s %s : "%(experiment,variable),no_real
    # feed_dic(case            ,d     ,experiment,variable,table,model,g,r,v)
    # feed_dic(selection       ,(g,r,v,p)   ,model)         # feed_dic(selection       ,(g,v,p)   ,model, real)
    # feed_dic(coverage_issues ,`d`    ,model,g,r)          # feed_dic(coverage_issues ,`d`    ,model,g,r,v)
    # feed_dic(holes           ,`holes`,model,grid,srindex)  #holes[version]=trous
    # no_real.append(model)
    return cases, selection, holes, coverage_issues, no_real

In [18]:
#a=select_models_data_versions("piControl","mrro","Lmon")
#allv,selected,holes,coverages,noreal=a
#len(selected)

In [19]:
#selected

In [20]:
#clog('error')
#a=select_models_data_versions("piControl","pr","day")
#allv,selected,holes,coverages,noreal=a
#len(selected)

In [17]:
#allv,selected,holes,coverages,noreal=a
#len(selected)

37

In [21]:
def clean_versions_dic_for_P_E(d):
    """
    Suppress those P-E entries in versions dict d which doesn't have a matching realization in matching evspsbl entries
    """
    def print_entries_number(exp,variable,table="Amon") :
        dic=d[exp][variable][table]
        count=0
        for model in dic :
            l=len(dic[model].keys())
            #print "%s %s,"%(model,l),
            if l>0 : count+=1
        print count
    #
    print "Counts of P-E before/after droping for missing evspsbl"
    for exp in d :
        if "P-E" not in d[exp] :
            continue
        print exp,
        print_entries_number(exp,"P-E"),"/",
        dPE=d[exp]["P-E"]["Amon"]
        for model in dPE.keys() :
            reals=dPE[model].keys()
            for real in reals :
                try:
                    real_evspsbl=d[exp]["evspsbl"]["Amon"][model][real]
                except:
                    #print "Droping P-E realization %s for %s"%(real,model)
                    dPE[model].pop(real)
            if len(dPE[model].keys()) == 0 :
                dPE.pop(model)
        #print exp,
        print_entries_number(exp,"P-E")

    return d
    
            

In [14]:
#clean=clean_versions_dic_for_P_E(read_versions_dic("20200719"))
#write_versions_dic(clean,"20200719d")

In [15]:
#clean=clean_versions_dic_for_P_E(read_versions_dic("20200719"))
#write_versions_dic(clean,"20200719c")

## Main function : creating a dictionary of data versions for a list of variables and experiments

In [22]:
sto=sys.stdout

In [23]:
#sys.stdout=sto

In [24]:
def create_versions_dictionnary(tag,experiments,variables,outfile=True,models=None):
    """
    Scrutinize available data on file system for :
    
      - a list of experiments, 
      - a dict of list of variables per table
      - a list of models (if provided) or all models
    
    Returns (and writes as json files) 5 dicts such as the 4 dicts and the list returned by  
    select_models_data_versions() (except for regularizing the dict structure to systematically 
    have keys : [experiment][variable][table][model] at the beginning)
    
    Please refer to that function for the details of those returned dicts : selection, cases, holes, 
    coverage_issues, no_realization
    
    Dict 'selection' is the main result, and documents all usable data for the grid, version and period covered
    
    We essentially loop calling select_models_data_versions() on experiments and variables per table, and 
    we add entries for a compound variable 'P-E' for each case where we have variable 'pr' in table 'Amon' 
    and also an entry for 'evspsbl' with the same realization index as 'pr'
    
    Dict 'cases' is the only one not not written as json file
    
    """
    #imposed={"CAMS-CSM1-0":"r2i1p1f1","EC-Earth3":"r4i1p1f1"}
    if outfile :
        import sys
        f = open(tag+".txt", 'w')
        orig_stdout = sys.stdout
        sys.stdout = f
    #
    select=dict() ; all_versions=dict() ; all_holes=dict(); all_coverages=dict(); noreals=dict()
    clog('error')
    for experiment in experiments:
        for table in variables :
             for variable in variables[table] :
                allv,selected,holes,coverages,noreal=select_models_data_versions(experiment,variable,table,models=models)
                feed_dic(all_versions  ,allv[experiment][variable][table],experiment,variable,table)
                feed_dic(select        ,selected ,experiment,variable,table)
                feed_dic(all_holes     ,holes    ,experiment,variable,table)
                feed_dic(all_coverages ,coverages,experiment,variable,table)
                feed_dic(noreals       ,noreal   ,experiment,variable,table)
                #
    #   
    # Maybe call check_realization_mismatch and chek_grid_mismatch here...
    if outfile :
        sys.stdout = orig_stdout
        f.close() 

    # Add entry for P-E by duplicating entry for pr and then removing entries which do not have 
    # the matching realization in evspsbl entries
    d=select
    for exp in d :
        if 'pr' in d[exp] and "Amon" in d[exp]['pr'] and 'P-E' not in d[exp]  :
                d[exp]['P-E']=dict()
                d[exp]['P-E']['Amon']= copy.deepcopy(d[exp]['pr']['Amon'])
    clean_versions_dic_for_P_E(d)
    
    # Write result as json
    write_versions_dic(select,tag)  
    write_versions_dic(all_holes,tag+"_holes")  
    write_versions_dic(all_coverages,tag+"_resolve")  
    write_versions_dic(noreals, tag+"_noreals")  
    return select, all_versions, all_holes, all_coverages, noreals

In [22]:
#dics = create_versions_dictionnary("2020911t",["historical"],{"day": ["pr"] },outfile=False)#,models=["UKESM1-0-LL"])
dics = create_versions_dictionnary("2020911t",["historical"],{"Amon": ["pr"] },outfile=False)#,models=["UKESM1-0-LL"])


Processing experiment historical, variable pr, table Amon
**************************************************
Identifying models that ran historical
UKESM1-0-LL          r12i1p1f2: no valid version among 1 : [u'v20191210'] 
HadGEM3-GC31-MM      r4i1p1f3: no valid version among 1 : [u'v20200601'] 
HadGEM3-GC31-MM      r2i1p1f3: no valid version among 1 : [u'v20191218'] 
HadGEM3-GC31-MM      r3i1p1f3: no valid version among 1 : [u'v20200601'] 
AWI-CM-1-1-MR        r2i1p1f1: no valid version among 1 : [u'v20200511'] 
AWI-CM-1-1-MR        r3i1p1f1: no valid version among 1 : [u'v20200511'] 
AWI-CM-1-1-MR        r4i1p1f1: no valid version among 1 : [u'v20200511'] 
AWI-CM-1-1-MR        r5i1p1f1: no valid version among 1 : [u'v20200511'] 
AWI-CM-1-1-MR        r1i1p1f1: no valid version among 1 : [u'v20200511'] 
no valid realization for  AWI-CM-1-1-MR historical pr Amon 18500101-20141230
AWI-ESM-1-1-LR       r1i1p1f1: no valid version among 1 : [u'v20200212'] 
no valid realization for  AWI-ESM

In [51]:
#dics=select, all_versions, all_holes, all_coverages, noreals

In [25]:
experiments=["piControl","historical","ssp126","ssp245","ssp585","ssp119"]
variables={
    "Amon": ["pr","tas","prw","evspsbl"], 
    "Lmon": ["mrro","mrso","mrsos"],
    "Omon": ["sos"],
    "day" : ["pr"]
    }

In [25]:
dics = create_versions_dictionnary("20200913",experiments,variables,outfile=False)#,models=["CNRM-CM6-1"])


Processing experiment piControl, variable pr, table day
**************************************************
Identifying models that ran piControl
MRI-ESM2-0           r1i2p1f1: no valid version among 1 : [u'v20200327'] 
MRI-ESM2-0           r1i1p1f1: no valid version among 1 : [u'v20190603'] 
no valid realization for  MRI-ESM2-0 piControl pr day *
AWI-ESM-1-1-LR       r1i1p1f1: no valid version among 1 : [u'v20200212'] 
no valid realization for  AWI-ESM-1-1-LR piControl pr day *


error    : u'Periods with holes are not handled [2219-2229, 2231-2314, 2316-2323, 2325-2419]'
error    : u'Periods with holes are not handled [2308, 2310, 2314, 2316, 2319, 2323-2324, 2327, 2332, 2337, 2341, 2344, 2346, 2348-2349, 2351, 2356, 2361, 2365-2366, 2368, 2370-2372, 2374, 2377-2378, 2380, 2382-2384, 2389-2390, 2394, 2396, 2399, 2402, 2405-2407, 2410-2411, 2416-2417, 2419-2424, 2428, 2430, 2432-2433, 2437-2438, 2440, 2443, 2447, 2449, 2453, 2463-2464, 2466-2469, 2471, 2478, 2482, 2485-2486, 2488, 2490, 2493, 2498-2499, 2502, 2506, 2508, 2510, 2517, 2522, 2526, 2529-2531, 2537, 2541, 2545, 2547-2548, 2559, 2562-2563, 2566, 2571-2572, 2574-2576, 2578-2579, 2587, 2591, 2594, 2597-2598, 2600-2602, 2607, 2611, 2621, 2623, 2626, 2629-2630, 2633-2634, 2642-2647, 2649-2651, 2654, 2661, 2664, 2668, 2671, 2676, 2681, 2684-2687, 2689, 2693, 2697, 2703, 2705, 2710, 2714-2716, 2720, 2723, 2727-2728, 2733, 2741, 2744, 2748, 2751, 2757, 2763, 2765, 2772, 2778, 2780, 2784, 2789, 2793, 2800]'


CNRM-CM6-1-HR        r1i1p1f2: no valid version among 1 : [u'v20191021'] 
no valid realization for  CNRM-CM6-1-HR piControl pr day *
IITM-ESM             r1i1p1f1: no valid version among 1 : [u'v20191120'] 
no valid realization for  IITM-ESM piControl pr day *
IPSL-CM6A-LR         r1i2p1f1: no valid version among 1 : [u'v20190319'] 


error    : u'Periods with holes are not handled [2015-2054, 2110-2150]'


models with no realization showing complete data for piControl pr :  ['HadGEM3-GC31-MM', 'MRI-ESM2-0', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'MCM-UA-1-0', 'CAMS-CSM1-0', 'E3SM-1-1', 'E3SM-1-0', 'E3SM-1-1-ECA', 'EC-Earth3-LR', 'EC-Earth3-Veg-LR', 'EC-Earth3-Veg', 'BCC-ESM1', 'BCC-CSM2-MR', 'NESM3', 'SAM0-UNICON', 'FIO-ESM-2-0', 'CNRM-CM6-1-HR', 'IITM-ESM', 'CIESM', 'NorESM1-F', 'NorCPM1', 'FGOALS-f3-L', 'CAS-ESM2-0', 'FGOALS-g3', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'GISS-E2-1-G', 'CanESM5-CanOE']

Processing experiment piControl, variable pr, table Amon
**************************************************
Identifying models that ran piControl


...

NorESM2-LM           r1i1p1f1: no valid version among 1 : [u'v20191108'] 
no valid realization for  NorESM2-LM ssp585 sos Omon 2015-2099
NorESM2-MM           r1i1p1f1: no valid version among 1 : [u'v20191108'] 
no valid realization for  NorESM2-MM ssp585 sos Omon 2015-2099
CESM2-WACCM          r3i1p1f1: no valid version among 1 : [u'v20200206'] 
CESM2-WACCM          r5i1p1f1: no valid version among 1 : [u'v20200206'] 
CESM2                r10i1p1f1: no valid version among 1 : [u'v20200528'] 
CESM2                r11i1p1f1: no valid version among 1 : [u'v20200528'] 
models with no realization showing complete data for ssp585 sos :  ['HadGEM3-GC31-MM', 'AWI-CM-1-1-MR', 'CAMS-CSM1-0', 'FIO-ESM-2-0', 'MPI-ESM1-2-LR', 'NorESM2-LM', 'NorESM2-MM', 'GFDL-ESM4']

Processing experiment ssp119, variable pr, table day
**************************************************
Identifying models that ran ssp119
EC-Earth3            r118i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3           

EC-Earth3            r143i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r126i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r110i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r119i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r107i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r142i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r121i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r111i1p1f1: no valid version among 1 : [u'v20200412'] 
no valid realization for  EC-Earth3 ssp119 pr Amon 2015-2099
EC-Earth3-Veg        r2i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r3i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r1i1p1f1: no valid version among 1 : [u'v20200226'] 
no valid realization for  EC-Earth3-Veg ssp119 pr Amon 2015-2099
FGOALS-g3       

EC-Earth3            r134i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r145i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r122i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r125i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r115i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r105i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r120i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r124i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r130i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r104i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r4i1p1f1: no valid version among 1 : [u'v20191015'] 
EC-Earth3            r108i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r143i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3     

EC-Earth3            r131i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r103i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r146i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r114i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r141i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r136i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r102i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r149i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r140i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r147i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r101i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r138i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r134i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3   

EC-Earth3            r113i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r139i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r135i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r132i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r106i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r116i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r129i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r137i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r109i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r133i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r127i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r117i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r128i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3   

In [26]:
dics = create_versions_dictionnary("20200918",experiments,variables,outfile=False)


Processing experiment piControl, variable pr, table day
**************************************************
Identifying models that ran piControl
MRI-ESM2-0           r1i2p1f1: no valid version among 1 : [u'v20200327'] 
MRI-ESM2-0           r1i1p1f1: no valid version among 1 : [u'v20190603'] 
no valid realization for  MRI-ESM2-0 piControl pr day *
AWI-ESM-1-1-LR       r1i1p1f1: no valid version among 1 : [u'v20200212'] 
no valid realization for  AWI-ESM-1-1-LR piControl pr day *


error    : u'Periods with holes are not handled [2219-2229, 2231-2314, 2316-2323, 2325-2419]'
error    : u'Periods with holes are not handled [2300-2440, 2442-2616, 2618-2672, 2674-2765, 2767-2772, 2774-2776, 2778-2800]'
error    : u'Periods with holes are not handled [1851-1888, 1890-1978, 1980-2064, 2067-2122, 2124-2134, 2136-2194, 2196-2257, 2259-2333, 2335-2349]'
error    : u'Periods with holes are not handled [0001-0019, 0021-0273, 02740102-07001231]'


CNRM-CM6-1-HR        r1i1p1f2: no valid version among 1 : [u'v20191021'] 
no valid realization for  CNRM-CM6-1-HR piControl pr day *
IITM-ESM             r1i1p1f1: no valid version among 1 : [u'v20191120'] 
no valid realization for  IITM-ESM piControl pr day *
IPSL-CM6A-LR         r1i2p1f1: no valid version among 1 : [u'v20190319'] 


error    : u'Periods with holes are not handled [2015-2054, 2110-2150]'


models with no realization showing complete data for piControl pr :  ['HadGEM3-GC31-MM', 'MRI-ESM2-0', 'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'MCM-UA-1-0', 'CAMS-CSM1-0', 'E3SM-1-1', 'E3SM-1-0', 'E3SM-1-1-ECA', 'EC-Earth3-LR', 'EC-Earth3-Veg-LR', 'EC-Earth3-Veg', 'BCC-ESM1', 'BCC-CSM2-MR', 'NESM3', 'SAM0-UNICON', 'FIO-ESM-2-0', 'CNRM-CM6-1-HR', 'IITM-ESM', 'CIESM', 'NorESM1-F', 'NorCPM1', 'FGOALS-f3-L', 'CAS-ESM2-0', 'FGOALS-g3', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'GISS-E2-1-G', 'CanESM5-CanOE']

Processing experiment piControl, variable pr, table Amon
**************************************************
Identifying models that ran piControl


...

EC-Earth3            r111i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3-Veg        r2i1p1f1: no valid version among 1 : [u'v20200226'] 
EC-Earth3-Veg        r3i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r6i1p1f1: no valid version among 1 : [u'v20200130'] 
EC-Earth3-Veg        r1i1p1f1: no valid version among 1 : [u'v20200225'] 
MPI-ESM1-2-LR        r10i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r3i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r7i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r9i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r4i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r5i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r6i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        r8i1p1f1: no valid version among 1 : [u'v20190710'] 
MPI-ESM1-2-LR        

EC-Earth3            r149i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r140i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r147i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r101i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r138i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r134i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r145i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r122i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r125i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r115i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r105i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r120i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r124i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3   

EC-Earth3            r133i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r127i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r117i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r128i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r150i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r148i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r131i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r103i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r146i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r114i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r141i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r136i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r102i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3   

EC-Earth3            r118i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r112i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r144i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r123i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r113i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r139i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r135i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r132i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r106i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r116i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r129i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r137i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3            r109i1p1f1: no valid version among 1 : [u'v20200412'] 
EC-Earth3   

EC-Earth3-Veg        r2i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r3i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r1i1p1f1: no valid version among 1 : [u'v20200226'] 
no valid realization for  EC-Earth3-Veg ssp119 mrso Lmon 2015-2099
models with no realization showing complete data for ssp119 mrso :  ['EC-Earth3', 'EC-Earth3-Veg']

Processing experiment ssp119, variable mrsos, table Lmon
**************************************************
Identifying models that ran ssp119
EC-Earth3-Veg        r2i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r3i1p1f1: no valid version among 2 : [u'v20191007', u'v20200515'] 
EC-Earth3-Veg        r1i1p1f1: no valid version among 1 : [u'v20200226'] 
no valid realization for  EC-Earth3-Veg ssp119 mrsos Lmon 2015-2099
models with no realization showing complete data for ssp119 mrsos :  ['EC-Earth3', 'EC-Earth3-Veg']

Processing experiment

In [21]:
noreals

{'piControl': {'sos': {'Omon': ['CanESM5-CanOE',
    'CNRM-CM6-1-HR',
    'CAMS-CSM1-0',
    'MRI-ESM2-0',
    'IITM-ESM',
    'GISS-E2-1-G-CC',
    'GISS-E2-1-H',
    'GISS-E2-2-G',
    'E3SM-1-1',
    'E3SM-1-1-ECA',
    'FIO-ESM-2-0',
    'MPI-ESM1-2-LR',
    'ACCESS-CM2',
    'ACCESS-ESM1-5',
    'CIESM',
    'FGOALS-f3-L',
    'CAS-ESM2-0',
    'FGOALS-g3',
    'NorESM1-F',
    'NorESM2-MM',
    'EC-Earth3-LR',
    'EC-Earth3',
    'EC-Earth3-Veg',
    'AWI-ESM-1-1-LR',
    'TaiESM1',
    'MPI-ESM-1-2-HAM']}}}

In [22]:
all_holes

{'piControl': {'sos': {'Omon': {'AWI-ESM-1-1-LR': {'gn': {'r1i1p1f1': {'v20200212': "ds('CMIP6%%sos%*%global%/bdd%AWI-ESM-1-1-LR%AWI%CMIP%Omon%piControl%r1i1p1f1%gn%v20200212')"}}},
    'CIESM': {'gn': {'r1i1p1f1': {'v20200220': "ds('CMIP6%%sos%*%global%/bdd%CIESM%THU%CMIP%Omon%piControl%r1i1p1f1%gn%v20200220')"}}},
    'CanESM5': {'gn': {'r1i1p2f1': {'v20190429': "ds('CMIP6%%sos%*%global%/bdd%CanESM5%CCCma%CMIP%Omon%piControl%r1i1p2f1%gn%v20190429')"}}},
    'CanESM5-CanOE': {'gn': {'r1i1p2f1': {'v20190429': "ds('CMIP6%%sos%*%global%/bdd%CanESM5-CanOE%CCCma%CMIP%Omon%piControl%r1i1p2f1%gn%v20190429')"}}},
    'EC-Earth3-Veg': {'gn': {'r1i1p1f1': {'v20200226': "ds('CMIP6%%sos%*%global%/bdd%EC-Earth3-Veg%EC-Earth-Consortium%CMIP%Omon%piControl%r1i1p1f1%gn%v20200226')"}}},
    'GISS-E2-1-G': {'gn': {'r1i1p1f2': {'v20190903': "ds('CMIP6%%sos%*%global%/bdd%GISS-E2-1-G%NASA-GISS%CMIP%Omon%piControl%r1i1p1f2%gn%v20190903')"},
      'r1i1p3f1': {'v20190710': "ds('CMIP6%%sos%*%global%/bdd%GISS

In [23]:
all_coverages

{'piControl': {'sos': {'Omon': {'CAS-ESM2-0': {'gn': {'r1i1p1f1': {'v20200306': '[0200-0399]'}}},
    'CNRM-CM6-1-HR': {'gn': {'r1i1p1f2': {'v20191021': '[1850-2149]'}}},
    'E3SM-1-1': {'gr': {'r1i1p1f1': {'v20191028': '[1850-2014]'}}},
    'FGOALS-f3-L': {'gn': {'r1i1p1f1': {'v20191028': '[0800-1099]'}}},
    'FGOALS-g3': {'gn': {'r1i1p1f1': {'v20191126': '[0400-0699]'}}},
    'GISS-E2-1-G': {'gn': {'r1i1p1f3': {'v20190916': '[2950-2999]'}}},
    'GISS-E2-1-G-CC': {'gn': {'r1i1p1f1': {'v20190815': '[1850-1950]'}}},
    'GISS-E2-1-H': {'gn': {'r1i1p3f1': {'v20191010': '[2000-2300]'}}},
    'IITM-ESM': {'gn': {'r1i1p1f1': {'v20191120': '[1926-2125]'}}},
    'IPSL-CM6A-LR': {'gn': {'r1i2p1f1': {'v20190319': '[1850-2099]'}}},
    'MRI-ESM2-0': {'gn': {'r1i2p1f1': {'v20200222': '[1850-2100]'}}},
    'NorESM1-F': {'gn': {'r1i1p1f1': {'v20190920': '[1501-1700]'}}}}}}}

## Exhibiting data holes or other coverage issue for models which ran the experiment but have no correct realization

In [54]:
def explain(variable, table, experiment,dics,
            print_select=True, print_holes=True, print_coverage=True, print_novar=True, explicit=True) :
    select, all_versions, all_holes, all_coverages, noreals = dics
    if explicit :
        prefix="%10s %10s %4s "%(experiment, variable, table)
    else : 
        prefix=""
    print
    if not explicit :print prefix+"\n"+30*"*"
    for model in select[experiment][variable][table]:
        grid,real,v,period=select[experiment][variable][table][model]
        if experiment != "piControl" : period=""
        if print_select :
            print "%-20s "%model+prefix+"OK       ",grid,real,v,period
    for model in noreals[experiment][variable][table]:
        reason_found=False
        try :
            for grid in all_holes[experiment][variable][table][model] :
                for real in all_holes[experiment][variable][table][model][grid]:
                    for v in all_holes[experiment][variable][table][model][grid][real] : 
                        a=eval(all_holes[experiment][variable][table][model][grid][real][v])
                        if print_holes : 
                            holes=`a.explore('choices')['period']`
                            if len(holes) > 80 : holes=holes[0:80]+"....."
                            print "%-20s "%model+prefix+"holes    ",grid,real,v,": ",holes
                        reason_found=True
                                
        except :
            pass 
        try :
            for grid in all_coverages[experiment][variable][table][model] :
                for real in all_coverages[experiment][variable][table][model][grid]:
                    for v in all_coverages[experiment][variable][table][model][grid][real] : 
                        a=all_coverages[experiment][variable][table][model][grid][real][v]
                        if print_coverage : 
                            print "%-20s "%model+prefix+"duration ",grid,real,v,": ",a
                        reason_found=True
        except :
            pass 
        if not reason_found :
            try :
                case=all_versions[experiment][variable][table][model] 
            except :
                if print_novar : 
                    print "%-20s "%model+prefix+"no_var"
                    
def explain_sorted(variable, table, experiment,dics,explicit=True) :
    explain(variable, table, experiment,dics, print_select=True, print_holes=False, print_coverage=False, print_novar=False, explicit=explicit)
    explain(variable, table, experiment,dics, print_select=False, print_holes=True, print_coverage=False, print_novar=False, explicit=explicit)
    explain(variable, table, experiment,dics, print_select=False, print_holes=False, print_coverage=True, print_novar=False, explicit=explicit)
    explain(variable, table, experiment,dics, print_select=False, print_holes=False, print_coverage=False, print_novar=True, explicit=explicit)

In [55]:
explain_sorted("sos","Omon","piControl",dics,explicit=True)


MPI-ESM1-2-HR         piControl        sos Omon OK        gn r1i1p1f1 v20190710 1850-2349
UKESM1-0-LL           piControl        sos Omon OK        gn r1i1p1f2 v20190827 1960-3059
HadGEM3-GC31-LL       piControl        sos Omon OK        gn r1i1p1f1 v20190628 1850-2349
NorCPM1               piControl        sos Omon OK        gn r1i1p1f1 v20190914 0001-0500
NorESM2-LM            piControl        sos Omon OK        gn r1i1p1f1 v20190920 1600-2100
IPSL-CM6A-LR          piControl        sos Omon OK        gn r1i1p1f1 v20200326 1850-3849
GISS-E2-1-G           piControl        sos Omon OK        gn r1i1p1f1 v20180824 4150-5000
CanESM5               piControl        sos Omon OK        gn r1i1p1f1 v20190429 5201-6200
NESM3                 piControl        sos Omon OK        gn r1i1p1f1 v20190704 1000-1499
CESM2-WACCM-FV2       piControl        sos Omon OK        gn r1i1p1f1 v20191120 0001-0500
INM-CM4-8             piControl        sos Omon OK        gr1 r1i1p1f1 v20190605 1850-2380
INM-CM5-

In [84]:
for experiment in experiments :
    for table in variables :
        for variable in variables[table] :
            explain_sorted(variable,table,experiment,dics,explicit=True)
            print 50*"*"


CNRM-ESM2-1           piControl         pr  day OK        gr r1i1p1f2 v20181115 1850-2349
IPSL-CM6A-LR          piControl         pr  day OK        gr r1i1p1f1 v20200326 1850-3849
GFDL-CM4              piControl         pr  day OK        gr2 r1i1p1f1 v20180701 0151-0650
CNRM-CM6-1            piControl         pr  day OK        gr r1i1p1f2 v20180814 1850-2349
NorESM2-LM            piControl         pr  day OK        gn r1i1p1f1 v20190920 1600-2100
INM-CM5-0             piControl         pr  day OK        gr1 r1i1p1f1 v20190619 1996-3196
MPI-ESM1-2-HR         piControl         pr  day OK        gn r1i1p1f1 v20190710 1850-2349
CESM2-WACCM           piControl         pr  day OK        gn r1i1p1f1 v20190320 0001-0500
CanESM5               piControl         pr  day OK        gn r1i1p2f1 v20190429 5550-6000
MIROC6                piControl         pr  day OK        gn r1i1p1f1 v20191016 3200-3699
CESM2-WACCM-FV2       piControl         pr  day OK        gn r1i1p1f1 v20191120 00010101-05010101

EC-Earth3-Veg         piControl        tas Amon holes     gr r1i1p1f1 v20200226 :  [1850-1861, 1863-1868, 1870-1904, 1906-1916, 1918-2024, 2026-2027, 2029-2065, 20.....

CNRM-CM6-1-HR         piControl        tas Amon duration  gr r1i1p1f2 v20191021 :  [1850-2149]
IITM-ESM              piControl        tas Amon duration  gn r1i1p1f1 v20191120 :  [1926-2125]
GISS-E2-1-G-CC        piControl        tas Amon duration  gn r1i1p1f1 v20190815 :  [1850-2014]
GISS-E2-2-G           piControl        tas Amon duration  gn r1i1p1f1 v20191120 :  [2000-2150]
E3SM-1-1              piControl        tas Amon duration  gr r1i1p1f1 v20191029 :  [1850-2014]
E3SM-1-1-ECA          piControl        tas Amon duration  gr r1i1p1f1 v20191216 :  [1850-2014]
NorESM1-F             piControl        tas Amon duration  gn r1i1p1f1 v20190920 :  [1501-1700]
AWI-ESM-1-1-LR        piControl        tas Amon duration  gn r1i1p1f1 v20200212 :  [1855-1954]

CanESM5-CanOE         piControl        tas Amon no_var
ACCESS-ESM1-5 

CNRM-CM6-1            piControl    evspsbl Amon holes     gr r1i1p1f2 v20180814 :  [2050-2240, 2261-2349]
GISS-E2-1-H           piControl    evspsbl Amon holes     gn r1i1p1f1 v20190410 :  [3231-3480, 3531-3580, 3681-3980]
GISS-E2-2-G           piControl    evspsbl Amon holes     gn r1i1p1f1 v20191120 :  [2000-2025, 2051-2125]
E3SM-1-1-ECA          piControl    evspsbl Amon holes     gr r1i1p1f1 v20191216 :  [1850-1929, 1940-2009]
MPI-ESM1-2-LR         piControl    evspsbl Amon holes     gn r1i1p1f1 v20190710 :  [1870-1889, 1910-1929, 2010-2029, 2050-2069, 2090-2109, 2130-2149, 2190-2229, 22.....
NorESM2-MM            piControl    evspsbl Amon holes     gn r1i1p1f1 v20191108 :  [1360-1369, 1500-1509, 1630-1639]
EC-Earth3-Veg         piControl    evspsbl Amon holes     gr r1i1p1f1 v20200226 :  [1852, 1859, 1872, 1874, 1877, 1879, 1884-1887, 1893-1894, 1896-1897, 1901-1902,.....
AWI-ESM-1-1-LR        piControl    evspsbl Amon holes     gn r1i1p1f1 v20200212 :  [1855-1856, 1858-1859, 1864

E3SM-1-1-ECA          piControl       mrso Lmon holes     gr r1i1p1f1 v20191218 :  [1940-1949, 2010-2014]
MPI-ESM1-2-LR         piControl       mrso Lmon holes     gn r1i1p1f1 v20190710 :  [1870-1889, 1950-1969, 2010-2049, 2230-2249, 2270-2309, 2330-2409, 2470-2509, 26.....
NorESM2-MM            piControl       mrso Lmon holes     gn r1i1p1f1 v20191108 :  [1220-1229, 1310-1319, 1360-1369, 1380-1389, 1460-1469, 1490-1499, 1550-1569, 16.....
EC-Earth3-Veg         piControl       mrso Lmon holes     gr r1i1p1f1 v20200226 :  [1850-1851, 1860, 1862, 1867, 1870, 1872-1874, 1878-1879, 1881, 1885, 1887, 1889.....
AWI-ESM-1-1-LR        piControl       mrso Lmon holes     gn r1i1p1f1 v20200212 :  [1857, 1859, 1869, 1871, 1874-1877, 1881, 1883, 1885-1887, 1889, 1894-1896, 1899.....
MPI-ESM-1-2-HAM       piControl       mrso Lmon holes     gn r1i1p1f1 v20190627 :  [1850-1869, 1890-1929, 2010-2029, 2050-2069, 2090-2129, 2210-2229, 2250-2269, 23.....

CNRM-CM6-1-HR         piControl       mrso Lmon 

GISS-E2-2-G           piControl        sos Omon holes     gn r1i1p1f1 v20191120 :  [2000-2075, 2101-2150]
MPI-ESM1-2-LR         piControl        sos Omon holes     gn r1i1p1f1 v20190710 :  [1910-1929, 1970-2009, 2070-2089, 2110-2129, 2150-2189, 2350-2389, 2410-2429, 25.....
CIESM                 piControl        sos Omon holes     gn r1i1p1f1 v20200220 :  [0001-0050, 0101-0150, 0201-0500]
NorESM2-MM            piControl        sos Omon holes     gn r1i1p1f1 v20191108 :  [1230-1239, 1250-1289, 1300-1309, 1340-1359, 1380-1389, 1410-1439, 1500-1519, 15.....
EC-Earth3-Veg         piControl        sos Omon holes     gn r1i1p1f1 v20200226 :  [1853, 1855, 1860, 1872, 1874-1876, 1881, 1885, 1887, 1889, 1894-1895, 1898, 191.....
AWI-ESM-1-1-LR        piControl        sos Omon holes     gn r1i1p1f1 v20200212 :  [1891-1910, 1921-1930]
MPI-ESM-1-2-HAM       piControl        sos Omon holes     gn r1i1p1f1 v20190627 :  [1890-1909, 1970-1989, 2030-2069, 2110-2129, 2150-2189, 2210-2229, 2250-2269, 22.

NorESM2-MM           historical       mrso Lmon duration  gn r1i1p1f1 v20191108 :  [1850-1859, 1880-1889, 1910-1919, 1930-1939, 1980-1989, 2000-2009]
TaiESM1              historical       mrso Lmon duration  gn r1i1p1f1 v20200218 :  [185002-201412]
MPI-ESM-1-2-HAM      historical       mrso Lmon duration  gn r2i1p1f1 v20190627 :  [1990-2014]
MPI-ESM-1-2-HAM      historical       mrso Lmon duration  gn r1i1p1f1 v20190627 :  [1850-1869, 1910-1929, 1970-1989]

NESM3                historical       mrso Lmon no_var
CanESM5-CanOE        historical       mrso Lmon no_var
GFDL-ESM4            historical       mrso Lmon no_var
IITM-ESM             historical       mrso Lmon no_var
GISS-E2-1-G-CC       historical       mrso Lmon no_var
E3SM-1-1-ECA         historical       mrso Lmon no_var
FIO-ESM-2-0          historical       mrso Lmon no_var
ACCESS-CM2           historical       mrso Lmon no_var
ACCESS-ESM1-5        historical       mrso Lmon no_var
CIESM                historical       mrso 

CNRM-CM6-1               ssp126      mrsos Lmon OK        gr r1i1p1f2 v20190219 
CNRM-CM6-1-HR            ssp126      mrsos Lmon OK        gr r1i1p1f2 v20200127 
MPI-ESM1-2-HR            ssp126      mrsos Lmon OK        gn r1i1p1f1 v20190710 
BCC-CSM2-MR              ssp126      mrsos Lmon OK        gn r1i1p1f1 v20190314 
MRI-ESM2-0               ssp126      mrsos Lmon OK        gn r1i1p1f1 v20191108 
ACCESS-CM2               ssp126      mrsos Lmon OK        gn r1i1p1f1 v20191108 
CESM2                    ssp126      mrsos Lmon OK        gn r1i1p1f1 v20190730 
FIO-ESM-2-0              ssp126      mrsos Lmon OK        gn r1i1p1f1 v20191227 
MIROC6                   ssp126      mrsos Lmon OK        gn r1i1p1f1 v20190627 
CESM2-WACCM              ssp126      mrsos Lmon OK        gn r1i1p1f1 v20190815 
UKESM1-0-LL              ssp126      mrsos Lmon OK        gn r1i1p1f2 v20190503 
GFDL-ESM4                ssp126      mrsos Lmon OK        gr1 r1i1p1f1 v20180701 
CanESM5                  ss

NorESM2-MM               ssp245      mrsos Lmon duration  gn r1i1p1f1 v20191108 :  [2021-2040]
HadGEM3-GC31-LL          ssp245      mrsos Lmon duration  gn r4i1p1f3 v20190908 :  [2015-2020]
HadGEM3-GC31-LL          ssp245      mrsos Lmon duration  gn r2i1p1f3 v20190906 :  [2015-2020]
HadGEM3-GC31-LL          ssp245      mrsos Lmon duration  gn r3i1p1f3 v20190908 :  [2015-2020]

NESM3                    ssp245      mrsos Lmon no_var
CanESM5-CanOE            ssp245      mrsos Lmon no_var
GISS-E2-1-G              ssp245      mrsos Lmon no_var
FIO-ESM-2-0              ssp245      mrsos Lmon no_var
ACCESS-CM2               ssp245      mrsos Lmon no_var
ACCESS-ESM1-5            ssp245      mrsos Lmon no_var
KACE-1-0-G               ssp245      mrsos Lmon no_var
AWI-CM-1-1-MR            ssp245      mrsos Lmon no_var
MCM-UA-1-0               ssp245      mrsos Lmon no_var
INM-CM5-0                ssp245      mrsos Lmon no_var
INM-CM4-8                ssp245      mrsos Lmon no_var
**************

GISS-E2-1-G              ssp585        sos Omon duration  gn r1i1p3f1 v20200115 :  [2051-2100]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r10i1p1f1 v20190710 :  [2035-2074]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r3i1p1f1 v20190710 :  [2015-2034]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r7i1p1f1 v20190710 :  [2055-2074, 2095-2100]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r9i1p1f1 v20190710 :  [2035-2054]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r4i1p1f1 v20190710 :  [2075-2094]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r5i1p1f1 v20190710 :  [2015-2034, 2095-2100]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r6i1p1f1 v20190710 :  [2075-2094]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r8i1p1f1 v20190710 :  [2055-2074, 2095-2100]
MPI-ESM1-2-LR            ssp585        sos Omon duration  gn r1i1p1f1 v20190710 :  [2055-2074]
NorESM2-LM      

In [36]:
exp='piControl'
for model in all_holes[exp]['sos']['Omon']: 
    for grid in all_holes[exp]['sos']['Omon'][model] :
        for real in all_holes[exp]['sos']['Omon'][model][grid] :
            for v in all_holes[exp]['sos']['Omon'][model][grid][real] :
                try:
                    a=eval(all_holes[exp]['sos']['Omon'][model][grid][real][v])
                    print(exp, model,grid,v,a.explore('choices')['period'])
                except : 
                    p=(exp, model,grid,v, a)
                    sys.stderr.write(`p`+"\n\n")


('piControl', 'GISS-E2-1-G', 'gn', 'v20190903', [7550-7600, 7651-7800])
('piControl', 'GISS-E2-1-G', 'gn', 'v20190710', [2000-2150, 2251-2350])
('piControl', 'CIESM', 'gn', 'v20200220', [0001-0050, 0101-0150, 0201-0450])
('piControl', 'GISS-E2-1-H', 'gn', 'v20190410', [3180-3280, 3331-3530, 3581-3780, 3881-3930])
('piControl', 'EC-Earth3-Veg', 'gn', 'v20200226', [1853, 1855, 1860, 1872, 1874-1876, 1881, 1885, 1887, 1889, 1894-1895, 1898, 1910, 1912, 1914, 1917, 1922, 1924, 1926-1928, 1930-1931, 1933, 1936, 1940-1941, 1943, 1949-1951, 1953, 1955-1956, 1961, 1964, 1969, 1971, 1974-1975, 1982-1983, 1985, 1994, 1998, 2004, 2009-2010, 2012, 2015, 2018, 2022, 2029, 2032-2035, 2039-2040, 2044, 2047, 2051, 2061, 2065, 2072, 2082-2083, 2087, 2089, 2093, 2095-2096, 2098-2099, 2101, 2104, 2108, 2111, 2122, 2126, 2129, 2139, 2141-2142, 2144, 2148-2150, 2154-2158, 2160-2161, 2164, 2167-2169, 2173, 2176, 2179, 2182-2187, 2193, 2199, 2208, 2210-2214, 2218-2219, 2222, 2224, 2226, 2231, 2245, 2252-2253

In [42]:
create_versions_dictionnary("20190218_pr",["piControl","historical","ssp126","ssp245","ssp585"],["pr"],outfile=True)

error    : 'Periods with holes are not handled [0500-0799, 0900-0999]'
error    : 'Periods with holes are not handled [3431-3480, 3531-3580, 3631-3830, 3881-3930]'
error    : 'Periods with holes are not handled [0001-0010, 0021-0040, 0061-0090, 0111-0130, 0141-0150, 0161-0180, 0191-0220, 0231-0240, 0251-0260, 0281-0300, 0331-0340, 0361-0400, 0411-0430, 0441-0450, 0461-0470, 0511-0530, 0541-0560, 0571-0580, 0611-0620, 0631-0650, 0661-0670, 0691-0700]'


Data versions dictionnary written as Data_versions_selection_20190218_pr.json


In [13]:
variables=["tas","pr","evspsbl","mrso","mrro","prw","sos","mrsos"]
experiments=["piControl","historical","ssp126","ssp245","ssp585","ssp119"]

In [46]:
if False :
    mod=set()
    for e in experiments :
        for m in list_CMIP6_models_that_run(e) :
            try :
                i=institute_for_model(m,mip_for_experiment(e))
            except:
                print m
                mod.add(m)
    print mod

Identifying models that ran piControl
Identifying models that ran historical
Identifying models that ran ssp126
Identifying models that ran ssp245
Identifying models that ran ssp585
Identifying models that ran ssp119
set([])


In [14]:
select, all_versions, resolve_issues, all_holes,  noreals = create_versions_dictionnary("20200519",experiments,variables,outfile=True)

error    : 'Periods with holes are not handled [1850-1900, 1951-2014]'
error    : 'Periods with holes are not handled [2000-2100, 2201-2300]'
error    : 'Periods with holes are not handled [2000-2050, 2101-2150, 2201-2250]'
error    : 'Periods with holes are not handled [1850-1900, 1951-2014]'
error    : 'Periods with holes are not handled [1850-1900, 1951-2014]'
error    : 'Periods with holes are not handled [7550-7600, 7651-7750, 7801-7894]'
error    : 'Periods with holes are not handled [1850-1879, 1890-1999]'
error    : 'Periods with holes are not handled [1990-2049, 2090-2109, 2130-2149, 2170-2229, 2270-2289, 2510-2529, 2550-2569, 2590-2609, 2630-2649, 2770-2789, 2830-2849]'
error    : 'Periods with holes are not handled [1240-1249, 1320-1329, 1340-1349, 1370-1379, 1410-1419, 1470-1479, 1510-1519, 1530-1539, 1640-1649, 1660-1669]'
error    : 'Periods with holes are not handled [1850, 1853, 1860, 1866, 1879, 1882, 1887, 1890, 1895, 1897-1898, 1909, 1927, 1932, 1934, 1941-1943, 1946

error    : 'Periods with holes are not handled [1850-1869, 1910-1949, 2030-2049, 2070-2089, 2110-2129, 2190-2209, 2330-2349, 2390-2449, 2510-2529, 2550-2569]'
error    : 'Periods with holes are not handled [0500-0699, 0800-0999]'
error    : 'Periods with holes are not handled [5550-6200, 6401-6600]'
error    : 'Periods with holes are not handled [3180-3280, 3331-3380, 3531-3580, 3631-3730, 3831-3880]'
error    : 'Periods with holes are not handled [2000-2050, 2101-2200]'
error    : 'Periods with holes are not handled [2000-2050, 2126-2150]'
error    : 'Periods with holes are not handled [2000-2050, 2251-2300]'
error    : 'Periods with holes are not handled [1880-1889, 1900-1919, 1930-1939, 1980-1999]'
error    : 'Periods with holes are not handled [1850-1869, 1970-1989, 2130-2209, 2350-2369, 2610-2629, 2770-2809]'
error    : 'Periods with holes are not handled [1250-1259, 1400-1409, 1460-1469, 1510-1519, 1530-1539]'
error    : 'Periods with holes are not handled [1850-1851, 1856, 1863,

Data versions dictionnary written as Data_versions_selection_20200519.json


In [32]:
help({}.pop)

Help on built-in function pop:

pop(...)
    D.pop(k[,d]) -> v, remove specified key and return the corresponding value.
    If key is not found, d is returned if given, otherwise KeyError is raised



In [63]:
for experiment in resolve_issues:
    for variable in resolve_issues[experiment] :
        for model in resolve_issues[experiment][variable] :
            for grid in resolve_issues[experiment][variable][model] :
                for real in resolve_issues[experiment][variable][model][grid]:
                    for version in resolve_issues[experiment][variable][model][grid][real] :
                        print experiment,model,variable,grid,real,version

piControl GISS-E2-1-G pr gn r1i1p1f2 v20190903
piControl GISS-E2-1-G pr gn r101i1p1f1 v20190815
piControl GISS-E2-1-G pr gn r1i1p3f1 v20190710
piControl GISS-E2-1-H pr gn r1i1p3f1 v20191010
piControl GISS-E2-1-H pr gn r1i1p1f1 v20190410
piControl E3SM-1-1-ECA pr gr r1i1p1f1 v20191216
piControl CanESM5 pr gn r1i1p2f1 v20190429
piControl AWI-ESM-1-1-LR pr gn r1i1p1f1 v20200212
piControl MPI-ESM1-2-LR pr gn r1i1p1f1 v20190710
piControl EC-Earth3-Veg pr gr r1i1p1f1 v20200226
piControl GISS-E2-2-G pr gn r1i1p1f1 v20191120
piControl NorESM2-LM pr gn r1i1p1f1 v20200217
piControl MPI-ESM-1-2-HAM pr gn r1i1p1f1 v20190627
piControl NorESM2-MM pr gn r1i1p1f1 v20191108
piControl GISS-E2-1-G tas gn r1i1p1f2 v20190903
piControl GISS-E2-1-G tas gn r102i1p1f1 v20190815
piControl GISS-E2-1-G tas gn r101i1p1f1 v20190815
piControl GISS-E2-1-G tas gn r1i1p3f1 v20190710
piControl GISS-E2-1-H tas gn r1i1p3f1 v20191010
piControl E3SM-1-1-ECA tas gr r1i1p1f1 v20191216
piControl MPI-ESM-1-2-HAM tas gn r1i1p1f1

In [52]:
#write_versions_dic(all_versions,"all_versions_20200519")
#write_versions_dic(all_holes,"all_holes_20200519")
#write_versions_dic(resolve_issues,"resolve_issues_20200519")
write_versions_dic(noreals,"noreals_20200519")

Data versions dictionnary written as Data_versions_selection_noreals_20200519.json


A vérifier :
- plutôt latest pour tous (sinon, voir errata de la version)
- P-E : 
  - choisir une realization commune entre pr et evspsbl
  - evspsbl=latest (pour calcul auto)
- signe de evspsbl

Superflu ? :
- meme r pour toutes expés
- mêmes i, p, f entre expériences

A lister :
- cas (e,v) où aucune version d'aucune realization n'est complète

Actuellement : 
- un passage avec selection de la version la plus récente sans trous
- puis verif des contraintes  -> realisations imposées


In [57]:
for k in noreals:
    print k
    for v in noreals[k]:
        print "\t%-8s %s"%( v, noreals[k][v])

ssp245
	pr       ['MPI-ESM1-2-LR', 'FGOALS-g3', 'NorESM2-LM', 'NorESM2-MM']
	tas      ['MPI-ESM1-2-LR', 'NorESM2-LM', 'NorESM2-MM']
	mrro     ['MPI-ESM1-2-LR', 'NorESM2-LM', 'NorESM2-MM']
	prw      ['MPI-ESM1-2-LR', 'FGOALS-g3', 'NorESM2-LM', 'NorESM2-MM']
	mrso     ['MPI-ESM1-2-LR', 'NorESM2-LM', 'NorESM2-MM']
	evspsbl  ['MPI-ESM1-2-LR', 'FGOALS-g3', 'NorESM2-LM', 'NorESM2-MM']
	mrsos    ['MPI-ESM1-2-LR', 'NorESM2-LM', 'NorESM2-MM', 'HadGEM3-GC31-LL']
	sos      ['MRI-ESM2-0', 'MPI-ESM1-2-LR', 'FGOALS-f3-L', 'NorESM2-LM', 'NorESM2-MM']
historical
	pr       ['GISS-E2-1-G-CC', 'E3SM-1-1-ECA', 'NorESM2-MM', 'MPI-ESM-1-2-HAM']
	tas      ['GISS-E2-1-G-CC', 'E3SM-1-1-ECA', 'NorESM2-MM', 'EC-Earth3-Veg', 'AWI-CM-1-1-MR', 'MPI-ESM-1-2-HAM']
	mrro     ['CESM2-WACCM-FV2', 'CESM2-FV2', 'E3SM-1-1', 'E3SM-1-1-ECA', 'NorESM2-MM', 'MPI-ESM-1-2-HAM']
	prw      ['GISS-E2-1-G-CC', 'E3SM-1-1-ECA', 'NorESM2-MM', 'MPI-ESM-1-2-HAM']
	mrso     ['E3SM-1-1', 'NorESM2-MM', 'TaiESM1', 'MPI-ESM-1-2-HAM']
	evspsbl

In [44]:
d=read_versions_dictionnary("20190218_pr")

In [31]:
if False :
    d=dict()
    clog('error')
    for experiment in experiments:
        d[experiment]=dict()
        for variable in variables :
            table="Amon"
            if variable in ["mrro","mrso","mrsos"] : table="Lmon"
            if variable in ["sos"] : table="Omon"
            d[experiment][variable]=select_models_data_versions(experiment,variable,table)
        

In [32]:
if False:
    d=dict()
    clog('error')
    variables=["pr","evspsbl","mrsos","mrro"]
    experiments=["ssp370"]
    for experiment in experiments:
        d[experiment]=dict()
        for variable in variables :
            table="Amon"
            if variable in ["mrro","mrso","mrsos"] : table="Lmon"
            d[experiment][variable]=select_models_data_versions(experiment,variable,table)
        

## Checking that realizations are the same for pr and evspsbl, for consistency in computing P-E
### (for versions, it is not the case, and will have to rely on link 'latest')

In [19]:
d=select
for experiment in experiments :
    print experiment
    for model in d[experiment]["pr"] :
        if model not in d[experiment]["evspsbl"] :
            print "\tno evspsbl for %s and %s"%(experiment,model)
        else :
            g,r,v,p=d[experiment]["pr"][model]
            gp,rp,vp,pp=d[experiment]["evspsbl"][model]
            if r != rp : 
                print '\tmismatch in realizations for %-20s'%model,r,rp
            else :
                if g != gp : 
                    print ' --mismatch in grid  for model',model,r,g,gp
                else :
                    if v != vp : 
                        pass
                        #print ' --mismatch in versions for model',model,r,v,vp

piControl
	no evspsbl for piControl and CNRM-CM6-1
historical
	mismatch in realizations for NorESM2-LM           r2i1p1f1 r1i1p1f1
	mismatch in realizations for GISS-E2-1-H          r2i1p1f1 r3i1p1f1
	no evspsbl for historical and FIO-ESM-2-0
	mismatch in realizations for CanESM5-CanOE        r3i1p2f1 r1i1p2f1
	mismatch in realizations for KACE-1-0-G           r2i1p1f1 r1i1p1f1
	mismatch in realizations for FGOALS-f3-L          r2i1p1f1 r1i1p1f1
ssp126
	mismatch in realizations for GISS-E2-1-G          r1i1p3f1 r1i1p1f2
	no evspsbl for ssp126 and FIO-ESM-2-0
	mismatch in realizations for CanESM5-CanOE        r1i1p2f1 r2i1p2f1
ssp245
	mismatch in realizations for GISS-E2-1-G          r2i1p3f1 r3i1p1f2
	no evspsbl for ssp245 and FIO-ESM-2-0
	no evspsbl for ssp245 and ACCESS-CM2
	mismatch in realizations for CanESM5-CanOE        r1i1p2f1 r3i1p2f1
ssp585
	no evspsbl for ssp585 and HadGEM3-GC31-LL
	mismatch in realizations for MRI-ESM2-0           r1i2p1f1 r1i1p1f1
	mismatch in realizations

In [38]:
def filter_for_evspsbl(d):
    for experiment in d :
        if "P-E" in d[experiment] :
            pr=d[experiment]["pr"]
            ev=d[experiment]["evspsbl"]
            models=d[experiment]["P-E"].keys()
            for model in models :
                if model not in pr or model not in ev : continue
                _,r_pr,_,_=pr[model]
                _,r_ev,_,_=ev[model]
                if r_pr != r_ev :
                    print "removing P-E for %s,%s (%s,%s)"%(experiment,model,r_pr,r_ev)
                    d[experiment]["P-E"].pop(model)
#
sel1=read_versions_dictionnary("20200519","./")
filter_for_evspsbl(sel1)
write_versions_dic(sel1,"20200519P-E")

removing evspsbl for ssp245,GISS-E2-1-G (r2i1p3f1,r3i1p1f2)
removing evspsbl for ssp245,CanESM5-CanOE (r1i1p2f1,r3i1p2f1)
removing evspsbl for historical,NorESM2-LM (r2i1p1f1,r1i1p1f1)
removing evspsbl for historical,GISS-E2-1-H (r2i1p1f1,r3i1p1f1)
removing evspsbl for historical,KACE-1-0-G (r2i1p1f1,r1i1p1f1)
removing evspsbl for historical,FGOALS-f3-L (r2i1p1f1,r1i1p1f1)
removing evspsbl for historical,CanESM5-CanOE (r3i1p2f1,r1i1p2f1)
removing evspsbl for ssp126,GISS-E2-1-G (r1i1p3f1,r1i1p1f2)
removing evspsbl for ssp126,CanESM5-CanOE (r1i1p2f1,r2i1p2f1)
removing evspsbl for ssp585,MRI-ESM2-0 (r1i2p1f1,r1i1p1f1)
removing evspsbl for ssp585,CanESM5-CanOE (r2i1p2f1,r3i1p2f1)
Data versions dictionnary written as Data_versions_selection_20200519P-E.json


## Checking if selected versions are the latest

In [27]:
print "Versions which are not the latest :"
for experiment in d:
    for variable in d[experiment] :
        if variable=="P-E" : continue
        for model in d[experiment][variable] :
            g,r,v,p=d[experiment][variable][model]
            table="Amon"
            if variable in ["mrro","mrso","mrsos"] : table="Lmon"
            if variable in ["sos"] : table="Omon"
            search="/bdd/CMIP6/%s/%s/%s/%s/%s/%s/%s/%s/*"%(
                mip_for_experiment(experiment),institute_for_model(model),model,experiment,r,table,variable,g)
            versions=glob.glob(search)
            versions=[ ve.split("/")[-1] for ve in versions ]
            versions.sort()
            if len(versions)==0 :
                print "For %s, %s and %s, no version with %s"%(experiment,variable,model,search)
                continue
            if v != versions[-1] :
                print "For %s, %s %s and %s, version is not the latest (%s,%s)"%(experiment,variable,model,r,v,versions[-1])
            #else :
            #    print "OK for %s, %s and %s, %s)"%(experiment,variable,model,v)
print "Done"

Versions which are not the latest :
For historical, tas EC-Earth3 and r16i1p1f1, version is not the latest (v20191020,v20200318)
Done


## Checking sign of evspsbl

In [28]:
means=dict()
for experiment in experiments:
    print experiment
    for variable in ["evspsbl"] :
        print "\t",variable
        for model in d[experiment][variable] :
            if "CAMS" in model or "EC-Earth" in model :
                g,r,v,p=d[experiment][variable][model]
                fyear=p.split('-')[0]
                table="Amon"
                #if variable in ["mrro","mrso"] : table="Lmon"
                dat=ds(project="CMIP6", experiment=experiment,model=model, institute="*",
                          period=fyear+"01", variable=variable, table=table, grid=g,
                          version=v, mip=mip_for_experiment(experiment),realization=r )
                mean=cvalue(ccdo(dat,operator="fldmean"))
                if model not in means : means[model]=dict()
                means[model][experiment]=mean
                if mean < 0. :
                    print"\t\t%20s"%model,r,v           

piControl
	evspsbl
		         CAMS-CSM1-0 r1i1p1f1 v20190729
historical
	evspsbl
		       EC-Earth3-Veg r4i1p1f1 v20190728
		         CAMS-CSM1-0 r1i1p1f2 v20191106
		           EC-Earth3 r4i1p1f1 v20190721
ssp126
	evspsbl
		         CAMS-CSM1-0 r1i1p1f1 v20190708
ssp245
	evspsbl
		       EC-Earth3-Veg r4i1p1f1 v20190915
		         CAMS-CSM1-0 r1i1p1f1 v20190708
		           EC-Earth3 r4i1p1f1 v20190809
ssp585
	evspsbl
		       EC-Earth3-Veg r4i1p1f1 v20190915
		         CAMS-CSM1-0 r1i1p1f1 v20190708
		           EC-Earth3 r4i1p1f1 v20190801
ssp119
	evspsbl
		         CAMS-CSM1-0 r1i1p1f1 v20190708


## Checking realization mismatch among experiments

In [30]:
def check_realizations_mismatch(d):
    mismatches=[]
    for variable in variables :
        print variable
        reals=dict()
        for experiment in experiments:
            if experiment=="piControl" :
                continue
            for model in d[experiment][variable] :
                g,r,v,p=d[experiment][variable][model]
                feed_dic(reals,r,model,experiment)
        for model in reals :
            rs=set(reals[model].values())
            if len(rs) > 1 :
                print "\t%20s has varied realizations "%(model),rs,reals[model]
                mismatches.append((model,variable))
    return mismatches
check_realizations_mismatch(select)        

tas
	          MRI-ESM2-0 has varied realizations  set(['r1i2p1f1', 'r1i1p1f1']) {'ssp126': 'r1i1p1f1', 'ssp119': 'r1i1p1f1', 'ssp585': 'r1i2p1f1', 'historical': 'r1i2p1f1', 'ssp245': 'r1i1p1f1'}
	         CAMS-CSM1-0 has varied realizations  set(['r1i1p1f2', 'r1i1p1f1']) {'ssp126': 'r1i1p1f1', 'ssp119': 'r1i1p1f1', 'ssp585': 'r1i1p1f1', 'historical': 'r1i1p1f2', 'ssp245': 'r1i1p1f1'}
	         GISS-E2-1-G has varied realizations  set(['r2i1p1f2', 'r1i1p1f2', 'r1i1p3f1']) {'ssp126': 'r1i1p3f1', 'historical': 'r1i1p1f2', 'ssp245': 'r2i1p1f2'}
	         FIO-ESM-2-0 has varied realizations  set(['r2i1p1f1', 'r1i1p1f1']) {'ssp126': 'r2i1p1f1', 'ssp585': 'r1i1p1f1', 'historical': 'r1i1p1f1', 'ssp245': 'r1i1p1f1'}
	           EC-Earth3 has varied realizations  set(['r4i1p1f1', 'r16i1p1f1']) {'ssp585': 'r4i1p1f1', 'historical': 'r16i1p1f1', 'ssp245': 'r4i1p1f1'}
pr
	          MRI-ESM2-0 has varied realizations  set(['r1i2p1f1', 'r1i1p1f1']) {'ssp126': 'r1i1p1f1', 'ssp119': 'r1i1p1f1', 'ssp585

[('MRI-ESM2-0', 'tas'),
 ('CAMS-CSM1-0', 'tas'),
 ('GISS-E2-1-G', 'tas'),
 ('FIO-ESM-2-0', 'tas'),
 ('EC-Earth3', 'tas'),
 ('MRI-ESM2-0', 'pr'),
 ('CAMS-CSM1-0', 'pr'),
 ('GISS-E2-1-G', 'pr'),
 ('FIO-ESM-2-0', 'pr'),
 ('HadGEM3-GC31-LL', 'pr'),
 ('FGOALS-f3-L', 'pr'),
 ('CanESM5-CanOE', 'pr'),
 ('CAMS-CSM1-0', 'evspsbl'),
 ('GISS-E2-1-G', 'evspsbl'),
 ('ACCESS-CM2', 'evspsbl'),
 ('CanESM5-CanOE', 'evspsbl'),
 ('CAMS-CSM1-0', 'mrso'),
 ('CAMS-CSM1-0', 'mrro'),
 ('FIO-ESM-2-0', 'mrro'),
 ('KACE-1-0-G', 'mrro'),
 ('MRI-ESM2-0', 'prw'),
 ('CAMS-CSM1-0', 'prw'),
 ('GISS-E2-1-G', 'prw'),
 ('CanESM5-CanOE', 'prw'),
 ('FGOALS-g3', 'sos'),
 ('MRI-ESM2-0', 'sos'),
 ('GISS-E2-1-G', 'sos'),
 ('FGOALS-f3-L', 'sos'),
 ('CanESM5-CanOE', 'sos'),
 ('CAMS-CSM1-0', 'mrsos')]

In [17]:
def check_grids_mismatch(d):
    mismatches=[]
    for variable in variables :
        print variable
        grids=dict()
        for experiment in experiments:
            if experiment=="piControl" :
                continue
            for model in d[experiment][variable] :
                g,r,v,p=d[experiment][variable][model]
                feed_dic(grids,g,model,experiment)
        for model in grids :
            rs=set(grids[model].values())
            if len(rs) > 1 :
                print "\t%20s has varied grids "%(model),rs,grids[model]
                mismatches.append((model,variable))
    return mismatches
        

- CAMS-CSM1-0 : hist f2, reste f1  -> Pas de Lmon dans hist r1..f1; imposer r2i1p1f1 partout (sauf piControl) (pas de r1..f2 dans les scenarios) ?
- EC-Earth3 : ssp119 r4, reste r1  -> imposer r4 partout (sauf piControl) ?
- INM-CM5-0, pour pr, evspsbl et prw (mais par pour Land): hist r2, reste r1 ; la table Lmon est dégarnie dans hist r1; il n'y a pas de scenarios autre que r1 (a part ssp370)

In [9]:
d2=dict()
clog('error')
for experiment in experiments:
    d2[experiment]=dict()
    for variable in variables :
        table="Amon"
        if variable in ["mrro","mrso"] : table="Lmon"
        if variable in ["sos"] : table="Omon"
        imposed={"CAMS-CSM1-0":"r2i1p1f1","EC-Earth3":"r4i1p1f1"}
        #imposed={"CAMS-CSM1-0":"r2i1p1f1"}
        d2[experiment][variable]=select_models_data_versions(experiment,variable,
                        table,imposed_realizations=imposed, models=imposed.keys())
        for model in imposed.keys() :
            if model in d[experiment][variable] :
                d[experiment][variable][model]=d2[experiment][variable][model]


Processing experiment piControl, variable tas
**************************************************
CAMS-CSM1-0          : 1 real(s), use r1i1p1f1/v20190729 (among 1 : ['v20190729'] ['r1i1p1f1']
EC-Earth3            : 1 real(s), use r1i1p1f1/v20190712 (among 1 : ['v20190712'] ['r1i1p1f1']

Processing experiment piControl, variable pr
**************************************************
CAMS-CSM1-0          : 1 real(s), use r1i1p1f1/v20190729 (among 1 : ['v20190729'] ['r1i1p1f1']
EC-Earth3            : 1 real(s), use r1i1p1f1/v20190712 (among 1 : ['v20190712'] ['r1i1p1f1']

Processing experiment piControl, variable evspsbl
**************************************************
CAMS-CSM1-0          : 1 real(s), use r1i1p1f1/v20190729 (among 1 : ['v20190729'] ['r1i1p1f1']
EC-Earth3            : 1 real(s), use r1i1p1f1/v20190712 (among 1 : ['v20190712'] ['r1i1p1f1']

Processing experiment piControl, variable mrso
**************************************************
CAMS-CSM1-0          : 1 real(s), 

EC-Earth3            : 1 real(s), use r4i1p1f1/v20190801 (among 1 : ['v20190801'] ['r4i1p1f1']

Processing experiment ssp585, variable evspsbl
**************************************************
CAMS-CSM1-0          : 1 real(s), use r2i1p1f1/v20190924 (among 1 : ['v20190924'] ['r2i1p1f1']
EC-Earth3            : 1 real(s), use r4i1p1f1/v20190801 (among 1 : ['v20190801'] ['r4i1p1f1']

Processing experiment ssp585, variable mrso
**************************************************
CAMS-CSM1-0          : 1 real(s), use r2i1p1f1/v20191212 (among 2 : ['v20190924', 'v20191212'] ['r2i1p1f1']
EC-Earth3            : 1 real(s), use r4i1p1f1/v20190801 (among 1 : ['v20190801'] ['r4i1p1f1']

Processing experiment ssp585, variable mrro
**************************************************
CAMS-CSM1-0          : 1 real(s), use r2i1p1f1/v20190924 (among 1 : ['v20190924'] ['r2i1p1f1']
EC-Earth3            : 1 real(s), use r4i1p1f1/v20190801 (among 1 : ['v20190801'] ['r4i1p1f1']

Processing experiment ssp585, 

# Update json file with choices for models with imposed realizations

In [18]:
for m,v in check_realizations_mismatch(d) :
    for e in experiments :
        if e in d and v in d[e] and m in d[e][v] :
            print e,v,m
            d[e][v].pop(m)

tas
pr
	           INM-CM5-0 has varied realizations  set([u'r2i1p1f1', u'r1i1p1f1']) {'ssp126': u'r1i1p1f1', 'ssp585': u'r1i1p1f1', 'historical': u'r2i1p1f1', 'ssp245': u'r1i1p1f1'}
	         FGOALS-f3-L has varied realizations  set([u'r2i1p1f1', u'r1i1p1f1']) {'ssp585': u'r1i1p1f1', 'historical': u'r2i1p1f1'}
evspsbl
	           INM-CM5-0 has varied realizations  set([u'r2i1p1f1', u'r1i1p1f1']) {'ssp126': u'r1i1p1f1', 'ssp585': u'r1i1p1f1', 'historical': u'r2i1p1f1', 'ssp245': u'r1i1p1f1'}
mrso
mrro
prw
	           INM-CM5-0 has varied realizations  set([u'r2i1p1f1', u'r1i1p1f1']) {'ssp126': u'r1i1p1f1', 'ssp585': u'r1i1p1f1', 'historical': u'r2i1p1f1', 'ssp245': u'r1i1p1f1'}
sos
	           INM-CM5-0 has varied realizations  set([u'r2i1p1f1', u'r1i1p1f1']) {'ssp126': u'r1i1p1f1', 'ssp585': u'r1i1p1f1', 'historical': u'r2i1p1f1', 'ssp245': u'r1i1p1f1'}
piControl pr INM-CM5-0
historical pr INM-CM5-0
ssp126 pr INM-CM5-0
ssp245 pr INM-CM5-0
ssp585 pr INM-CM5-0
piControl pr FGOALS-f3-L
h

In [19]:
write_versions_dic("20190217")

## A function for listing models for a set of experiments and a variable

In [77]:
def check_models_for_experiments(dic,variable,experiments):
    l=models_for_experiments(d,variable,experiments)
    l=sorted(l)
    print len(l)#
    #print l
    for m in l :
        ref=None
        issue=False
        print "% 15s"%m,
        for e in experiments :
            g,r,v,p=d[e][variable][m]
            if e == "piControl":
                print "%-3s %s %s" % (g,r,p.split("-")[0]),
            else:
                if ref is None :
                    ref=r
                elif ref != r :
                    issue=True
                print "%-3s %s " % (g,r),
        if issue : 
            print "<----- Some realizations differ (apart from piControl)!!",
        print
    return l

In [76]:
check_models_for_experiments(d,"pr",["piControl","historical","ssp585",])

21
    BCC-CSM2-MR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
    CAMS-CSM1-0 gn  r1i1p1f1 2900 gn  r2i1p1f1  gn  r2i1p1f1 
          CESM2 gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
    CESM2-WACCM gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
     CNRM-CM6-1 gr  r1i1p1f2 1850 gr  r1i1p1f2  gr  r1i1p1f2 
    CNRM-ESM2-1 gr  r1i1p1f2 1850 gr  r1i1p1f2  gr  r1i1p1f2 
        CanESM5 gn  r1i1p2f1 5550 gn  r1i1p1f1  gn  r1i1p1f1 
      EC-Earth3 gr  r1i1p1f1 2259 gr  r4i1p1f1  gr  r4i1p1f1 
  EC-Earth3-Veg gr  r1i1p1f1 1850 gr  r1i1p1f1  gr  r1i1p1f1 
    FGOALS-f3-L gr  r1i1p1f1 0600 gr  r2i1p1f1  gr  r1i1p1f1  <----- Some realizations differ !!
       GFDL-CM4 gr1 r1i1p1f1 0151 gr1 r1i1p1f1  gr1 r1i1p1f1 
      GFDL-ESM4 gr1 r1i1p1f1 0001 gr1 r1i1p1f1  gr1 r1i1p1f1 
      INM-CM4-8 gr1 r1i1p1f1 1850 gr1 r1i1p1f1  gr1 r1i1p1f1 
      INM-CM5-0 gr1 r1i1p1f1 1996 gr1 r1i1p1f1  gr1 r1i1p1f1 
   IPSL-CM6A-LR gr  r1i1p1f1 1850 gr  r1i1p1f1  gr  r1i1p1f1 
     MCM-UA-1-0 gn  r1i1p1f1 000

['BCC-CSM2-MR',
 'CAMS-CSM1-0',
 'CESM2',
 'CESM2-WACCM',
 'CNRM-CM6-1',
 'CNRM-ESM2-1',
 'CanESM5',
 'EC-Earth3',
 'EC-Earth3-Veg',
 'FGOALS-f3-L',
 'GFDL-CM4',
 'GFDL-ESM4',
 'INM-CM4-8',
 'INM-CM5-0',
 'IPSL-CM6A-LR',
 'MCM-UA-1-0',
 'MIROC-ES2L',
 'MIROC6',
 'MPI-ESM1-2-HR',
 'MRI-ESM2-0',
 'UKESM1-0-LL']

In [21]:
check_models_for_experiments(d,"pr",["piControl","historical","ssp245",])

18
[u'BCC-CSM2-MR', u'CAMS-CSM1-0', u'CESM2', u'CESM2-WACCM', u'CNRM-CM6-1', u'CNRM-ESM2-1', u'CanESM5', u'EC-Earth3', u'EC-Earth3-Veg', u'GFDL-CM4', u'GFDL-ESM4', u'HadGEM3-GC31-LL', u'IPSL-CM6A-LR', u'MCM-UA-1-0', u'MIROC-ES2L', u'MIROC6', u'MRI-ESM2-0', u'UKESM1-0-LL']
    BCC-CSM2-MR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
    CAMS-CSM1-0 gn  r1i1p1f1 2900 gn  r2i1p1f1  gn  r2i1p1f1 
          CESM2 gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
    CESM2-WACCM gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
     CNRM-CM6-1 gr  r1i1p1f2 1850 gr  r1i1p1f2  gr  r1i1p1f2 
    CNRM-ESM2-1 gr  r1i1p1f2 1850 gr  r1i1p1f2  gr  r1i1p1f2 
        CanESM5 gn  r1i1p1f1 5201 gn  r1i1p1f1  gn  r1i1p1f1 
      EC-Earth3 gr  r1i1p1f1 2259 gr  r4i1p1f1  gr  r4i1p1f1 
  EC-Earth3-Veg gr  r1i1p1f1 1850 gr  r1i1p1f1  gr  r1i1p1f1 
       GFDL-CM4 gr1 r1i1p1f1 0151 gr1 r1i1p1f1  gr1 r1i1p1f1 
      GFDL-ESM4 gr1 r1i1p1f1 0001 gr1 r1i1p1f1  gr1 r1i1p1f1 
HadGEM3-GC31-LL gn  r1i1p1f1 1850 gn  r1i1p1f

In [13]:
def select_realization_and_version(cases,selection,coverage_issues,model,experiment,variable,table,period,project): 
    """
    Scrutinize all datasets in cases[MODEL][GRID][REALIZATION][VERSION], selecting the prefered 
    grid for MODEL (if applicable), then looping across realization (with priority for lower r index), 
    and then across versions (with prioriy to the newest) for identifying a realization+version pair which 
    datafiles covers PERIOD 
    
    So the choice in dic SELECTION is the newest version which has correct data coverage, in the realization 
    with smaller r indexv
    
    Pairs with uncomplete data coverage are stored in COVERAGE_ISSUES, but only until (in the double loop) a 
    correct pair is found
    
    NOTE : using this function is deprecated
    """
    # all combinations which data files actually covers the requested period
    # with one single grid+realization+version per model
    grids=cases[experiment][variable][table][model].keys()
    if len(grids) > 1 : 
        gr=None
        if model in preferred_grids : 
            gr=preferred_grids[model]
            if type(gr)==type([]) :
                gr=set(gr).intersection(set(grids))
                if len(gr)==1 :
                    gr=gr.pop()
                else :
                    gr=None
        if gr is None or gr not in grids :
            print "!!! multiple grids for ",model,experiment, variable, table, ":",grids
            raise ValueError("!!! multiple grids for %s %s %s %s : %s"%(model,experiment, variable, table,grids))
    else :
        gr=grids[0]
    if True:
        reals=cases[experiment][variable][table][model][gr].keys()
        sreals=[r for r in reals]
        ok=False
        while len(reals)>0 and not ok :
            # Want the smallest 'r' index, then the newest version for this index
            rindex=1000
            for r in reals :
                ri=r[1:].split("i")[0]
                if int(ri) < rindex : 
                    rindex=int(ri)
                    srindex=r
            reals.remove(srindex)
            #
            versions=cases[experiment][variable][table][model][grids[0]][srindex].keys()
            versions.sort()
            sversions=[ v for v in versions ]
            while len(versions)>0 and not ok :
                holes=dict()
                last=versions[-1]
                # Check available period is OK 
                d=ds(project=project, experiment=experiment,model=model, institute=institute_for_model(model),
                          period="*", variable=variable, table=table, 
                          version=last, mip=mip_for_experiment(experiment),realization=srindex, grid=grids[0] )
                data_period=d.explore('choices',operation='union').get('period',None)
                speriod=data_period
                #print "model=",model,"data_period=",data_period,type(data_period),type(data_period[0])
                if data_period is not None : 
                    data_period=data_period[0]
                    if period != "*" :
                        if data_period.includes(climaf.classes.init_period(period)) :
                            ok=True
                            actual_period=data_period
                        else : 
                            holes[last]=`speriod`
                            versions.remove(last)
                    else :
                        if experiment != "piControl" :
                            raise ValueError("Cannot process period * except for piControl")
                        # we relax the AR5 criterion of 100 years for spinup followed by 400 years
                        # by suppressing the 100 years criterion, and even allow only 399 years
                        if (data_period.end - data_period.start) >= 399*datetime.timedelta(days=365) :
                            ok=True
                            actual_period=data_period
                        else : 
                            holes[last]=`speriod`
                            versions.remove(last)
                if not ok :
                    #print "*** %-20s : issue with %s , no version covers the period (%s); "%(model,srindex, holes)
                    feed_dic(coverage_issues,holes,model,gr,srindex)
        if ok :
            #print "%-20s : %d real(s), use %s/%s (among %s : %s %s"%(model,len(sreals),srindex, last, len(versions),versions,sreals)
            feed_dic(selection,(grids[0],srindex,last,data_period.pr()),model)
            return(grids[0],srindex,last,data_period.pr())
        else :
            print "no selection for ",model,experiment,variable,table,period

In [9]:
def filter_and_order_realizations():
    """
    Provided with a dict of (realizations,version,grid,period), and for a given model, experiment and variable , 
    returns an ordered list across realizations for this case, where some may have been discarded base on 
    external criteria such as realizations description on ES-doc or CMIP6 errata
    
    The order is based on indices r and i of the variant : lowest i is preferable over lowest r 
    """
    pass # Not yet developped. Useful ?

In [22]:
check_models_for_experiments(d,"pr",["piControl","historical","ssp585",])

17
[u'BCC-CSM2-MR', u'CAMS-CSM1-0', u'CESM2', u'CESM2-WACCM', u'CNRM-CM6-1', u'CNRM-ESM2-1', u'CanESM5', u'EC-Earth3', u'EC-Earth3-Veg', u'GFDL-CM4', u'GFDL-ESM4', u'IPSL-CM6A-LR', u'MCM-UA-1-0', u'MIROC-ES2L', u'MIROC6', u'MRI-ESM2-0', u'UKESM1-0-LL']
    BCC-CSM2-MR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
    CAMS-CSM1-0 gn  r1i1p1f1 2900 gn  r2i1p1f1  gn  r2i1p1f1 
          CESM2 gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
    CESM2-WACCM gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
     CNRM-CM6-1 gr  r1i1p1f2 1850 gr  r1i1p1f2  gr  r1i1p1f2 
    CNRM-ESM2-1 gr  r1i1p1f2 1850 gr  r1i1p1f2  gr  r1i1p1f2 
        CanESM5 gn  r1i1p1f1 5201 gn  r1i1p1f1  gn  r1i1p1f1 
      EC-Earth3 gr  r1i1p1f1 2259 gr  r4i1p1f1  gr  r4i1p1f1 
  EC-Earth3-Veg gr  r1i1p1f1 1850 gr  r1i1p1f1  gr  r1i1p1f1 
       GFDL-CM4 gr1 r1i1p1f1 0151 gr1 r1i1p1f1  gr1 r1i1p1f1 
      GFDL-ESM4 gr1 r1i1p1f1 0001 gr1 r1i1p1f1  gr1 r1i1p1f1 
   IPSL-CM6A-LR gr  r1i1p1f1 1850 gr  r1i1p1f1  gr  r1i1p1f1 
   

## List models producing sos for ssp585, historical, and piControl 

In [20]:
check_models_for_experiments(d,"sos",["piControl","historical","ssp585",])

16
[u'BCC-CSM2-MR', u'CESM2', u'CESM2-WACCM', u'CNRM-CM6-1', u'CNRM-ESM2-1', u'CanESM5', u'EC-Earth3', u'EC-Earth3-Veg', u'GFDL-CM4', u'INM-CM4-8', u'IPSL-CM6A-LR', u'MCM-UA-1-0', u'MIROC-ES2L', u'MIROC6', u'NESM3', u'UKESM1-0-LL']
    BCC-CSM2-MR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
          CESM2 gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
    CESM2-WACCM gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
     CNRM-CM6-1 gn  r1i1p1f2 1850 gn  r1i1p1f2  gn  r1i1p1f2 
    CNRM-ESM2-1 gn  r1i1p1f2 1850 gn  r1i1p1f2  gn  r1i1p1f2 
        CanESM5 gn  r1i1p1f1 5201 gn  r1i1p1f1  gn  r1i1p1f1 
      EC-Earth3 gn  r1i1p1f1 2259 gn  r1i1p1f1  gn  r1i1p1f1 
  EC-Earth3-Veg gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
       GFDL-CM4 gn  r1i1p1f1 0151 gn  r1i1p1f1  gn  r1i1p1f1 
      INM-CM4-8 gr1 r1i1p1f1 1850 gr1 r1i1p1f1  gr1 r1i1p1f1 
   IPSL-CM6A-LR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
     MCM-UA-1-0 gn  r1i1p1f1 0001 gn  r1i1p1f2  gn  r1i1p1f2 
     MIROC-ES2L gn  r1i1

In [21]:
check_models_for_experiments(d,"sos",["piControl","historical","ssp245",])

16
[u'BCC-CSM2-MR', u'CESM2', u'CESM2-WACCM', u'CNRM-CM6-1', u'CNRM-ESM2-1', u'CanESM5', u'EC-Earth3', u'EC-Earth3-Veg', u'GFDL-CM4', u'HadGEM3-GC31-LL', u'IPSL-CM6A-LR', u'MCM-UA-1-0', u'MIROC-ES2L', u'MIROC6', u'NESM3', u'UKESM1-0-LL']
    BCC-CSM2-MR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
          CESM2 gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
    CESM2-WACCM gn  r1i1p1f1 0001 gn  r1i1p1f1  gn  r1i1p1f1 
     CNRM-CM6-1 gn  r1i1p1f2 1850 gn  r1i1p1f2  gn  r1i1p1f2 
    CNRM-ESM2-1 gn  r1i1p1f2 1850 gn  r1i1p1f2  gn  r1i1p1f2 
        CanESM5 gn  r1i1p1f1 5201 gn  r1i1p1f1  gn  r1i1p1f1 
      EC-Earth3 gn  r1i1p1f1 2259 gn  r1i1p1f1  gn  r1i1p1f1 
  EC-Earth3-Veg gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
       GFDL-CM4 gn  r1i1p1f1 0151 gn  r1i1p1f1  gn  r1i1p1f1 
HadGEM3-GC31-LL gn  r1i1p1f1 1850 gn  r1i1p1f3  gn  r1i1p1f3 
   IPSL-CM6A-LR gn  r1i1p1f1 1850 gn  r1i1p1f1  gn  r1i1p1f1 
     MCM-UA-1-0 gn  r1i1p1f1 0001 gn  r1i1p1f2  gn  r1i1p1f2 
     MIROC-ES2L gn